### Top selling Products

In [0]:
%sql
SELECT
    p.product_nm,
    SUM(f.sales) AS total_sales,
    SUM(f.quantity)     AS total_quantity
FROM bike_data.gold.fact_sales f
JOIN bike_data.gold.dim_products p
     ON f.product_key =
   CONCAT_WS(
     '-',
     SLICE(
       SPLIT(p.product_key, '-'),
       SIZE(SPLIT(p.product_key, '-')) - 2,
       3
     )
   )
GROUP BY p.product_nm
ORDER BY total_sales DESC;


Databricks visualization. Run in Databricks to view.

In [0]:
spark.table("bike_data.gold.dim_customers").columns

### Top Customers

In [0]:
%sql
SELECT
    c.customer_id,
    CONCAT(c.customer_firstname, ' ', c.customer_lastname) AS customer_name,
    c.country,
    SUM(f.sales) AS total_revenue,
    COUNT(DISTINCT f.order_number) AS total_orders
FROM bike_data.gold.fact_sales f
JOIN bike_data.gold.dim_customers c
    ON f.customer_id = c.customer_id
GROUP BY
    c.customer_id,
    c.customer_firstname,
    c.customer_lastname,
    c.country
ORDER BY total_revenue DESC
LIMIT 10;


### Top Customers by Country

In [0]:
%sql
SELECT
    c.country,
    c.customer_id,
    CONCAT(c.customer_firstname, ' ', c.customer_lastname) AS customer_name,
    SUM(f.sales) AS total_revenue
FROM bike_data.gold.fact_sales f
JOIN bike_data.gold.dim_customers c
    ON f.customer_id = c.customer_id
WHERE    c.country IS NOT NULL AND TRIM(c.country) <> ''   
GROUP BY c.country, c.customer_id, c.customer_firstname, c.customer_lastname
ORDER BY country, total_revenue DESC;


### Revenue by country

In [0]:
%sql
select 
      c.country,
      sum(f.sales)
from bike_data.gold.fact_sales f
join bike_data.gold.dim_customers c
on f.customer_id = c.customer_id
where c.country is not null and trim(c.country) <> ''
group by c.country
order by sum(f.sales) desc     

### Top Categories by revenue

In [0]:
spark.table("bike_data.gold.dim_products").columns

In [0]:
%sql
select 
      p.category,
      sum(f.sales) as revenue
from bike_data.gold.fact_sales f
join bike_data.gold.dim_products p
on f.product_key =    CONCAT_WS(
     '-',
     SLICE(
       SPLIT(p.product_key, '-'),
       SIZE(SPLIT(p.product_key, '-')) - 2,
       3
     )
   )
group by p.category
order by revenue desc


### Gender Wise Revenue

In [0]:
%sql
select 
      c.customer_gndr as Gender,
      sum(f.sales) as revenue
from bike_data.gold.fact_sales f
join bike_data.gold.dim_customers c
on f.customer_id = c.customer_id
group by c.customer_gndr
order by revenue desc

In [0]:
spark.table('bike_data.gold.dim_customers').printSchema()
spark.table('bike_data.gold.dim_products').printSchema()
spark.table('bike_data.gold.fact_sales').printSchema()


### Detailed View

In [0]:
%sql
select 
      concat(c.customer_firstname,' ',c.customer_lastname) as customer,
      c.country,
      p.category,
      p.product_key,
      p.product_cost,
      p.product_nm,
      p.subcategory,
      f.order_dt,
      count(f.quantity) as quantity,
      sum(f.sales) as revenue
from bike_data.gold.dim_customers c
join bike_data.gold.fact_sales f
     on c.customer_id =f.customer_id
join bike_data.gold.dim_products p
     on f.product_key =    CONCAT_WS(
     '-',
     SLICE(
       SPLIT(p.product_key, '-'),
       SIZE(SPLIT(p.product_key, '-')) - 2,
       3
     )
   )  
group by concat(c.customer_firstname,' ',c.customer_lastname),c.country,p.category,p.product_key,p.product_cost,p.product_nm,p.subcategory,f.order_dt 
order by revenue desc    

### High Revenue but Low Volume Products :  Which products sell less but earn more per unit?

In [0]:
%sql
SELECT
    p.product_nm,
    SUM(f.sales) AS total_sales,
    SUM(f.quantity) AS total_quantity,
    SUM(f.sales) / SUM(f.quantity) AS avg_price
FROM bike_data.gold.fact_sales f
JOIN bike_data.gold.dim_products p
  ON f.product_key =
     CONCAT_WS(
       '-',
       SLICE(
         SPLIT(p.product_key, '-'),
         SIZE(SPLIT(p.product_key, '-')) - 2,
         3
       )
     )
GROUP BY p.product_nm
ORDER BY avg_price DESC;
